In [1]:
import pandas as pd
import numpy as np
import glob

from matplotlib import pyplot as plt
import seaborn as sns

import re
import string

from tqdm import tqdm


import torch
import tensorflow as tf
import transformers

import networkx as nx
from node2vec import Node2Vec

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report

import gensim

## Merging all csv files

In [2]:
filenames = glob.glob("russian-troll-tweets/*.csv")

li = []
for filename in filenames:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    
dfs = pd.concat(li, axis=0, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (0,15,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (10,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
dfs.head()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,2535818742,HAPPKENDRAHAPPY,Bosh situation hasn't changed since Feb. Heat ...,United States,English,9/23/2016 17:08,9/23/2016 17:08,1311,1688,2285,...,Right,1,RightTroll,0,2535818742,779366817752084481,http://twitter.com/happkendrahappy/statuses/77...,NaN,NaN,NaN
1,2535818742,HAPPKENDRAHAPPY,Youre an IDIOT! Now you say 99% when before yo...,United States,English,9/23/2016 17:08,9/23/2016 17:08,1311,1688,2284,...,Right,1,RightTroll,0,2535818742,779366807446642688,http://twitter.com/happkendrahappy/statuses/77...,https://twitter.com/erecordscity/status/779354...,NaN,NaN
2,2535818742,HAPPKENDRAHAPPY,Charlotte-Mecklenburg Fraternal Order of Polic...,United States,English,9/23/2016 17:08,9/23/2016 17:08,1311,1688,2286,...,Right,1,RightTroll,0,2535818742,779366828585910272,http://twitter.com/happkendrahappy/statuses/77...,http://www.breitbart.com,NaN,NaN
3,2535818742,HAPPKENDRAHAPPY,Theodore Roosevelt's son Quentin and his frien...,United States,English,9/23/2016 17:09,9/23/2016 17:09,1311,1688,2290,...,Right,1,RightTroll,0,2535818742,779367073998835712,http://twitter.com/happkendrahappy/statuses/77...,https://twitter.com/HistoryInPics/status/73882...,NaN,NaN
4,2535818742,HAPPKENDRAHAPPY,.@flashfire451: Are there more cures than term...,United States,English,9/23/2016 17:09,9/23/2016 17:09,1311,1688,2289,...,Right,1,RightTroll,0,2535818742,779367063479521281,http://twitter.com/happkendrahappy/statuses/77...,NaN,NaN,NaN


In [4]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2946207 entries, 0 to 2946206
Data columns (total 21 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   external_author_id  object
 1   author              object
 2   content             object
 3   region              object
 4   language            object
 5   publish_date        object
 6   harvested_date      object
 7   following           int64 
 8   followers           int64 
 9   updates             int64 
 10  post_type           object
 11  account_type        object
 12  retweet             int64 
 13  account_category    object
 14  new_june_2018       int64 
 15  alt_external_id     object
 16  tweet_id            int64 
 17  article_url         object
 18  tco1_step1          object
 19  tco2_step1          object
 20  tco3_step1          object
dtypes: int64(6), object(15)
memory usage: 472.0+ MB


In [5]:
dfs['account_type'].value_counts()

Right         711668
Russian       704917
local         459220
Left          427141
Hashtager     241786
news          139006
Commercial    121904
German         91511
Italian        15680
?              13539
Koch           10894
Arabic          6228
Spanish         1226
French          1117
ZAPOROSHIA       175
Portuguese       118
Ebola             71
Ukranian           4
Uzbek              2
Name: account_type, dtype: int64

In [6]:
dfs = dfs[['author', 'content', 'account_category']].loc[dfs['account_category'].isin(['RightTroll', 'LeftTroll', 'NewsFeed'])]

### Drop NaN

In [7]:
dfs.dropna(how='any', subset=['author', 'content'], inplace=True)
print(dfs.shape)

(1737035, 3)


In [8]:
dfs['account_category'].value_counts()

RightTroll    711668
NewsFeed      598226
LeftTroll     427141
Name: account_category, dtype: int64

In [9]:
dfs = dfs.reset_index(drop=True)
dfs

,author,content,account_category
0,HAPPKENDRAHAPPY,Bosh situation hasn't changed since Feb. Heat ...,RightTroll
1,HAPPKENDRAHAPPY,Youre an IDIOT! Now you say 99% when before yo...,RightTroll
2,HAPPKENDRAHAPPY,Charlotte-Mecklenburg Fraternal Order of Polic...,RightTroll
3,HAPPKENDRAHAPPY,Theodore Roosevelt's son Quentin and his frien...,RightTroll
4,HAPPKENDRAHAPPY,.@flashfire451: Are there more cures than term...,RightTroll
...,...,...,...
1737030,_SOLOMONALBERT_,These police are train killers. I don’t shed a...,LeftTroll
1737031,_SOLOMONALBERT_,#BUT - @FloydMayweather Retires From #Boxing ...,LeftTroll
1737032,_SOLOMONALBERT_,No guns allowed to all people. Especially #bla...,LeftTroll
1737033,_SOLOMONALBERT_,Support the #petition that banns providing #bl...,LeftTroll


## Taking 200,000 row for each 'account_category'

In [174]:
# RightTroll = dfs[dfs['account_category'] == 'RightTroll'][:200_000]
# NewsFeed = dfs[dfs['account_category'] == 'NewsFeed'][:200_000]
# LeftTroll = dfs[dfs['account_category'] == 'LeftTroll'][:200_000]
# filterd_df = pd.concat([RightTroll, NewsFeed, LeftTroll])
# filterd_df = filterd_df.reset_index(drop=True)

# Using all
filterd_df = dfs.copy()

In [175]:
filterd_df.drop_duplicates(subset=['author'])['account_category'].value_counts()

RightTroll    630
LeftTroll     233
NewsFeed       54
Name: account_category, dtype: int64

In [176]:
filterd_df['account_category'].value_counts()

RightTroll    711668
NewsFeed      598226
LeftTroll     427141
Name: account_category, dtype: int64

In [177]:
filterd_df

,author,content,account_category
0,HAPPKENDRAHAPPY,Bosh situation hasn't changed since Feb. Heat ...,RightTroll
1,HAPPKENDRAHAPPY,Youre an IDIOT! Now you say 99% when before yo...,RightTroll
2,HAPPKENDRAHAPPY,Charlotte-Mecklenburg Fraternal Order of Polic...,RightTroll
3,HAPPKENDRAHAPPY,Theodore Roosevelt's son Quentin and his frien...,RightTroll
4,HAPPKENDRAHAPPY,.@flashfire451: Are there more cures than term...,RightTroll
...,...,...,...
1737030,_SOLOMONALBERT_,These police are train killers. I don’t shed a...,LeftTroll
1737031,_SOLOMONALBERT_,#BUT - @FloydMayweather Retires From #Boxing ...,LeftTroll
1737032,_SOLOMONALBERT_,No guns allowed to all people. Especially #bla...,LeftTroll
1737033,_SOLOMONALBERT_,Support the #petition that banns providing #bl...,LeftTroll


# U2H & U2M

### Creating 'hashtags' and 'mentions'  columns

### temp_df with 'author' and 'content'

In [178]:
temp_df = filterd_df[['author', 'content']]
temp_df.head()

,author,content
0,HAPPKENDRAHAPPY,Bosh situation hasn't changed since Feb. Heat ...
1,HAPPKENDRAHAPPY,Youre an IDIOT! Now you say 99% when before yo...
2,HAPPKENDRAHAPPY,Charlotte-Mecklenburg Fraternal Order of Polic...
3,HAPPKENDRAHAPPY,Theodore Roosevelt's son Quentin and his frien...
4,HAPPKENDRAHAPPY,.@flashfire451: Are there more cures than term...


### 'hashtags' column

- #### Collecting hashtags from content

In [179]:
temp_df['hashtags'] = filterd_df['content'].str.findall(r'#.*?(?=\s|$)')
temp_df.head()

,author,content,hashtags
0,HAPPKENDRAHAPPY,Bosh situation hasn't changed since Feb. Heat ...,[]
1,HAPPKENDRAHAPPY,Youre an IDIOT! Now you say 99% when before yo...,[]
2,HAPPKENDRAHAPPY,Charlotte-Mecklenburg Fraternal Order of Polic...,[]
3,HAPPKENDRAHAPPY,Theodore Roosevelt's son Quentin and his frien...,[]
4,HAPPKENDRAHAPPY,.@flashfire451: Are there more cures than term...,[]


### 'mentions' column

- #### Collecting mentions from content

In [180]:
temp_df['mentions'] = filterd_df['content'].str.findall(r'@.*?(?=\s|$)')
temp_df.head()

,author,content,hashtags,mentions
0,HAPPKENDRAHAPPY,Bosh situation hasn't changed since Feb. Heat ...,[],[]
1,HAPPKENDRAHAPPY,Youre an IDIOT! Now you say 99% when before yo...,[],[]
2,HAPPKENDRAHAPPY,Charlotte-Mecklenburg Fraternal Order of Polic...,[],[]
3,HAPPKENDRAHAPPY,Theodore Roosevelt's son Quentin and his frien...,[],[]
4,HAPPKENDRAHAPPY,.@flashfire451: Are there more cures than term...,[],[@flashfire451:]


# Graph requirements

- U2H
 + dictionary contains {'author' : list of all his unique hashtags}
 + List of unique authors
 + List of unique hashtags
 
- U2M
 + dictionary contains {'author' : list of all his unique mentions}
 + List of unique authors
 + List of unique mentions

## U2H

#### - Creating dictionary contains {'author' : list of all his unique hashtags}

In [181]:
hashtags_dic = dict()
for i in tqdm(range(temp_df.shape[0]), position=0):
    hashtags_dic.setdefault(temp_df.iloc[i, 0], []).append(temp_df.iloc[i, 2])

100%|██████████| 1737035/1737035 [01:35<00:00, 18196.97it/s]


In [182]:
print("author: ", list(hashtags_dic.keys())[0])
print("\nhashtags: ", list(hashtags_dic.values())[0])
print("\n# hashtags:", len(list(hashtags_dic.values())[0]))

author:  HAPPKENDRAHAPPY

hashtags:  [[], [], [], [], [], [], [], [], [], ['#ThingsMoreTrustedThanHillary'], ['#ThingsMoreTrustedThanHillary'], ['#ThingsMoreTrustedThanHillary'], ['#ThingsMoreTrustedThanHillary'], ['#ThingsMoreTrustedThanHillary'], ['#ThingsMoreTrustedThanHillary'], ['#ThingsMoreTrustedThanHillary'], ['#UNGA'], [], ['#ThursdayThoughts', '#life'], [], [], ['#win'], ['#CleanYourOwn'], [], ['#TrumpsArmy', '#TrumpTrain', '#Trump', '#NeverHillary', '#CrookedHillary', '#VoteTrump', '#USA', '#MAGA', '#MASA', '#MakeAmericaGreatAgain', '#military', '#Vets', '#military'], [], ['#HobokenTrainCrash', '#Islamist'], ['#NewJersey', '#HobokenTrainCrash'], [], [], [], [], [], [], [], [], ['#Benghazi', '#ClintonScandals'], [], [], ['#BlackLivesMatter'], [], [], [], [], [], [], [], [], ['#Hillary', '#ThursdayThoughts'], [], [], [], ['#debatenight', '#SamanthaBee'], ['#DebateNight', '#TrumpPence16', '#Trump'], [], [], ['#1'], ['#CometLanding'], [], [], ['#ClintonScandals'], [], ['#thefive

#### - Flatting the hashtags lists for each author

In [183]:
for i, (k, v) in enumerate(hashtags_dic.items()):
    hashtags_dic[k] = set([item for sublist in list(hashtags_dic.values())[i] for item in sublist])

In [184]:
print("author: ", list(hashtags_dic.keys())[0])
print("\nhashtags: ", list(hashtags_dic.values())[0])
print("\n# hashtags:", len(list(hashtags_dic.values())[0]))

author:  HAPPKENDRAHAPPY

hashtags:  {'#Pathetic.', '#ukraine', '#pizza', '#rkba', '#1010Means', '#VoteTrumpSaveAmerica', '#MSNBC', '#Elizabeth', '#StopCISA', '#NeverMitt', '#infosec', '#ASMSG', '#CatholicPriests', '#PodestaEmails25', '#Colorado:', '#TobiasBeecher', '#Lucas:', '#tco...', '#veterans', '#fakenews', '#HappyBirthdayGeorgeWashington', '#iamamuslimtoo', '#Christmas', '#nbc', '#Rohingya', '#DrainTheMediaSwamp', '#governmentshutdown', '#Constitution', '#Russian', '#CCoT', '#variety', '#TheStrain', '#WarOnWomen', '#SubmarineDay,', '#MichaelMoore', '#prolife', '#assclown', '#violent', '#ICE', '#Netflix', '#Resistance', '#prayforBrussels', '#Scientology', '#Democrats', '#Hannity', '#Conagra', '#Merkel.', "#FeelTheBern'", '#backtheblue', '#NoBigDeal?', '#trump2016', '#filmmatters', '#Dems', '#Left', '#KochBrothers,', '#Tcot', '#InvestigateShadowGovernment', '#HOUequality', '#JamesComey', '#fbm15?', '#Washington', '#Christian,', '#conscience', '#abortion', '#UPDATE', '#facesofobama

#### - Creating a list with all unique hashtags

In [185]:
hashtags_list = [item for sublist in list(temp_df['hashtags']) for item in sublist]

In [186]:
print(len(hashtags_list))
unique_hashtags_list = set(hashtags_list)
print(len(unique_hashtags_list))

1209966
101922


## U2M

#### - Creating dictionary contains {'author' : list of all his unique mentions}

In [187]:
mentions_dic = dict()
for i in tqdm(range(temp_df.shape[0]), position=0):
    mentions_dic.setdefault(temp_df.iloc[i, 0], []).append(temp_df.iloc[i, 3])

100%|██████████| 1737035/1737035 [01:34<00:00, 18391.11it/s]


In [188]:
print("author: ", list(mentions_dic.keys())[0])
print("\nhashtags: ", list(mentions_dic.values())[0])
print("\n# hashtags:", len(list(mentions_dic.values())[0]))

author:  HAPPKENDRAHAPPY

hashtags:  [[], [], [], [], ['@flashfire451:'], [], [], [], [], [], [], [], [], [], [], [], [], ['@FoxNews'], [], [], [], ['@PowerATL!'], ['@HillaryClinton'], [], [], [], [], [], [], ['@CoraSherlock'], ['@1_Hoof_Hearted', '@Merry__Can', '@CloydRivers'], ['@RMKitchens', '@1_Hoof_Hearted', '@NBCNews'], ['@ReportUnited', '@1_Hoof_Hearted'], ['@ReportUnited', '@1_Hoof_Hearted'], [], [], ['@HillaryClinton'], ['@maddow!'], [], [], [], [], [], ['@dailycaller'], ['@greeneyes0084', '@BobMacAZ', '@HillaryClinton'], [], ['@YouTube'], [], [], ['@JoeSchoffstall'], [], ['@RobHoey'], [], ['@EvilHillaryPics'], ['@kevkid79', '@worldnetdaily'], ['@Crossbowman1', "@mayday2744'"], [], ["@ESA_Rosetta's"], [], [], [], ['@POTUS'], [], [], [], [], [], [], [], [], [], [], [], [], [], ['@CNN'], ['@ShidelerK'], [], [], ['@worldnetdaily'], [], ['@SenatorSessions'], ['@TocRadio'], [], [], [], [], [], [], [], ['@MrGrantReynolds.'], [], [], ['@ProgressOutlook'], ['@realDonaldTrump'], ['@Pol

#### - Flatting the mentions lists for each author

In [189]:
for i, (k, v) in enumerate(mentions_dic.items()):
    mentions_dic[k] = set([item for sublist in list(mentions_dic.values())[i] for item in sublist])

In [190]:
print("author: ", list(mentions_dic.keys())[0])
print("\nhashtags: ", list(mentions_dic.values())[0])
print("\n# hashtags:", len(list(mentions_dic.values())[0]))

author:  HAPPKENDRAHAPPY

hashtags:  {'@DTyler51', '@DaWoggly', '@gregghoulden:', '@svdate', '@JoseD89344985', '@BlackrobePrecha', '@sdv92:', '@Tommythespider', '@HillaryClinton', '@andygrammer', '@seanhannity', "@Reince's", '@SalenaZito', '@GaryByrneAuthor', '@TweetRyanReeves', '@timesofisrael', '@A_M_Perez', '@ErinBurnett', '@SStantis', '@TenNamesLater', '@barbandaaron', '@Bournemouthecho', '@electionland', '@politicususa', '@LSUJEFF', '@SenateMajLdr', '@alexstubb', '@HillaryforNH', '@pjamesjp1', '@MinuteMenNews', '@JAmy208:', '@ACUConservative', '@PeterTownsend7:', '@hrkbenowen', '@darcyelizbet', '@chuckwoolery', '@DanScavino', '@noltenc?', '@CarmineZozzora', '@FLOTUS', '@WeThePeople1st', '@redrightvideos', '@PBS', '@MMFlint', '@realDonaldTrump→', '@wjmcgurn:And', '@GSDDoglover', '@GatorsFB', '@ATHudd', '@MariaBonanno', '@support', '@voxdotcom', '@bonniebo40:', "@FearDotA's", '@JoeHodas', '@Johnny_Petrini', '@kingattax:', '@Senwarren', '@presbo', '@GrainOfSands', '@julieroginsky☭', 

#### - Creating a list with all unique mentions

In [191]:
mentions_list = [item for sublist in list(temp_df['mentions']) for item in sublist]

In [192]:
print(len(mentions_list))
unique_mentions_list = set(mentions_list)
print(len(unique_mentions_list))

416020
102973


# Contructing U2H and U2M graphs

In [193]:
import networkx as nx
U2H = nx.Graph()
U2M = nx.Graph()

### Adding nodes

#### -  'author' nodes

In [194]:
U2H.add_nodes_from(hashtags_dic.keys())
U2M.add_nodes_from(mentions_dic.keys())

#### - 'hashtags' and 'mentions' nodes

In [195]:
U2H.add_nodes_from(unique_hashtags_list)
U2M.add_nodes_from(unique_mentions_list)

### Creating edges

#### - U2H edges

In [196]:
for i, (k, v) in enumerate(hashtags_dic.items()):
    for h in v:
        U2H.add_edge(k, h)

#### - U2M edges

In [197]:
for i, (k, v) in enumerate(mentions_dic.items()):
    for h in v:
        U2M.add_edge(k, h)

# Fitting U2H and U2M graphs
We give node2vec.Node2Vec a networkx.Graph instance, and after using .fit() (which accepts any parameter accepted by we get a gensim.models.Word2Vec) we get in return a gensim.models.Word2Vec instance.

## U2H

### - Creating node2vec instance

In [104]:
# Generate walks
node2vec = Node2Vec(U2H, dimensions=128, walk_length=10, num_walks=10)

Generating walks (CPU: 1): 100%|██████████| 10/10 [14:09<00:00, 84.96s/it]


### - Fitting the model

In [110]:
# Learn embeddings 
U2H_model = node2vec.fit(window=10, min_count=1)

In [64]:
U2H_model.save("U2H_model600k/U2H_model600k.model")

In [198]:
U2H_model = gensim.models.Word2Vec.load("U2H_model/U2H_model.model")

### - Inspect the similarity

In [160]:
for node, _ in U2H_model.most_similar('#Saudi'):
    print(node)

#Yemen
#UK
#Aleppo
#whitegenocide
#gunsense
#SanFrancisco
#BDS
#WearHoodieForTrayvon
#UAE
#EU


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


### Creating a dictionary `U2H_dic` {'author':vec}

In [199]:
U2H_dic = dict()
for i, (k, v) in enumerate(hashtags_dic.items()):
    U2H_dic[k] = U2H_model.wv.get_vector(k)

In [200]:
df_unique_U2H = filterd_df[['author', 'account_category']].drop_duplicates()
df_unique_U2H['vec'] = list(U2H_dic.values())
df_unique_U2H

,author,account_category,vec
0,HAPPKENDRAHAPPY,RightTroll,"[0.92348844, -4.314939, 0.11164494, 1.5373602,..."
67,HARERETRT,RightTroll,"[-0.0038229772, 0.00071280595, 0.0023527432, -..."
73,HARRYMAGICIAN_,RightTroll,"[-1.2162895, -0.8329594, 0.5094632, -1.6895013..."
123,HARRYTHEFATHER,RightTroll,"[-1.0077562, -0.7404025, -0.5766445, -0.384885..."
245,HATTIEB78,RightTroll,"[-1.4926754, -1.408123, 0.17445554, -0.5077729..."
...,...,...,...
1732275,_NICKLUNA_,RightTroll,"[1.9208452, -0.21148059, 0.35299873, 0.3450843..."
1736520,_RONBEN,RightTroll,"[-0.6838968, -1.1236953, -0.08839527, -0.09788..."
1736590,_RUBY_WILLS_,RightTroll,"[-1.8777606, -2.4200456, 1.4245462, -1.8028708..."
1736930,_SHERYLGILBERT,RightTroll,"[-0.50279295, -1.172616, -0.48094288, 0.451466..."


### Logistic regression

In [201]:
X_U2H = list(df_unique_U2H['vec'])
y = df_unique_U2H['account_category']

X_train, X_test, y_train, y_test = train_test_split(X_U2H, y, test_size=0.1, random_state=0)

In [202]:
lg = LogisticRegression(max_iter=1000)
lg.fit(X_train,y_train)
y_pred = lg.predict(X_test)

In [203]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   LeftTroll       0.88      0.85      0.87        27
    NewsFeed       0.78      0.78      0.78         9
  RightTroll       0.89      0.91      0.90        56

    accuracy                           0.88        92
   macro avg       0.85      0.85      0.85        92
weighted avg       0.88      0.88      0.88        92



## U2M

### - Creating node2vec instance

In [117]:
# Generate walks
U2M_node2vec = Node2Vec(U2M, dimensions=128, walk_length=10, num_walks=10)

Generating walks (CPU: 1): 100%|██████████| 10/10 [15:31<00:00, 93.19s/it]


### - Fitting the model

In [118]:
# Learn embeddings 
U2M_model = U2M_node2vec.fit(window=10, min_count=1)

### - Inspect the similarity

In [210]:
for node, _ in U2M_model.most_similar('@FoxNews'):
    print(node)

@HillaryClinton
@POTUS
@CNN
@GOP
@nytimes
@YouTube
@washingtonpost
@CBSNews
@ABC
@BernieSanders


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


In [63]:
U2M_model.save("U2M_model600k.model")

In [204]:
U2M_model = gensim.models.Word2Vec.load("U2M_model/U2M_model.model")

### Creating a dictionary `U2M_dic` {'author':vec}

In [205]:
U2M_dic = dict()
for i, (k, v) in enumerate(mentions_dic.items()):
    U2M_dic[k] = U2M_model.wv.get_vector(k)

In [206]:
df_unique_U2M = filterd_df[['author', 'account_category']].drop_duplicates()
df_unique_U2M['vec'] = list(U2M_dic.values())
df_unique_U2M

,author,account_category,vec
0,HAPPKENDRAHAPPY,RightTroll,"[0.43625987, -3.4778433, 0.80007875, -0.288606..."
67,HARERETRT,RightTroll,"[-0.0038229772, 0.00071280595, 0.0023527432, -..."
73,HARRYMAGICIAN_,RightTroll,"[0.5914636, -1.4669645, -0.5887222, 0.5088199,..."
123,HARRYTHEFATHER,RightTroll,"[0.49560964, -2.688947, -1.5042355, 1.4916233,..."
245,HATTIEB78,RightTroll,"[-0.26410463, -1.5123643, -0.6777114, -0.17407..."
...,...,...,...
1732275,_NICKLUNA_,RightTroll,"[0.95913476, -1.7549795, 0.10077851, -3.674983..."
1736520,_RONBEN,RightTroll,"[-0.0055986233, -2.40265, -1.3271798, 1.050968..."
1736590,_RUBY_WILLS_,RightTroll,"[-1.3231806, -2.9713647, -5.873148, 2.6422498,..."
1736930,_SHERYLGILBERT,RightTroll,"[0.6025622, -0.15346095, -0.57556415, 0.191819..."


### Logistic regression

In [207]:
X_U2M = list(df_unique_U2M['vec'])
y = df_unique_U2M['account_category']

X_train, X_test, y_train, y_test = train_test_split(X_U2M, y, test_size=0.1, random_state=0)

In [208]:
lg = LogisticRegression(max_iter=1000)
lg.fit(X_train,y_train)
y_pred = lg.predict(X_test)

In [209]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   LeftTroll       0.86      0.67      0.75        27
    NewsFeed       0.80      0.44      0.57         9
  RightTroll       0.83      0.98      0.90        56

    accuracy                           0.84        92
   macro avg       0.83      0.70      0.74        92
weighted avg       0.84      0.84      0.82        92



# BERT

In [40]:
import torch
import torch.nn as nn

from transformers import AutoModel, AutoTokenizer, AutoConfig

In [156]:
MODEL_NAME = "bert-large-uncased"
config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

In [157]:
device = torch.device('cuda')
model = model.to(device)

In [158]:
bert_df = filterd_df.copy()
bert_df.head()

,author,content,account_category
0,HAPPKENDRAHAPPY,Bosh situation hasn't changed since Feb. Heat ...,RightTroll
1,HAPPKENDRAHAPPY,Youre an IDIOT! Now you say 99% when before yo...,RightTroll
2,HAPPKENDRAHAPPY,Charlotte-Mecklenburg Fraternal Order of Polic...,RightTroll
3,HAPPKENDRAHAPPY,Theodore Roosevelt's son Quentin and his frien...,RightTroll
4,HAPPKENDRAHAPPY,.@flashfire451: Are there more cures than term...,RightTroll


In [ ]:
x = []
for line in filterd_df['content']:
    x.append(len(line.strip().split(' ')))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
np.random.seed(42)
# x = np.random.normal(size=1000)
plt.hist(x, density=True, bins=20)  # `density=False` would make counts
plt.ylabel('Probability')
plt.xlabel('Data');

## BERT Pre-Processing
`tokenizer.encode(sent, add_special_tokens=True, padding='max_length')`: This turns every sentence into the list of ids with [CLS], [SEP] and [PAD] tokens .


In [ ]:
tqdm.pandas()
bert_df['content'] = bert_df['content'].progress_apply(
    tokenizer.encode,
    add_special_tokens=True,
    padding=True,
    max_length=64,
    return_tensors='pt')

In [ ]:
bert_df

In [ ]:
input_ids = bert_df['content']

In [ ]:
model.eval()
pooler_output = [None] * len(input_ids)
for i in tqdm(range(len(input_ids))):
    tensor_gpu = input_ids[i].to(device) # Moving tensor to the GPU
    with torch.no_grad():
        pooler_output[i] = model(tensor_gpu)[0][:,0,:].cpu().numpy() # Taking CLS tensors
        del tensor_gpu
        torch.cuda.empty_cache()

## Creating `bert_df_output` with `pooler_output`

In [ ]:
bert_df_output = bert_df.copy()
bert_df_output['content'] = pooler_output

In [ ]:
bert_df_output

## Faltten all vectors

In [135]:
bert_df_output['content'] = list(np.concatenate(bert_df_output['content'].values))

In [138]:
bert_df_output

,author,content,account_category
0,HAPPKENDRAHAPPY,"[-0.9964574, -0.98038715, 0.9999991, -0.998856...",RightTroll
1,HAPPKENDRAHAPPY,"[-0.9999339, -0.9988255, 0.99999946, -0.996039...",RightTroll
2,HAPPKENDRAHAPPY,"[-0.99954855, -0.9936008, 0.9999993, -0.996664...",RightTroll
3,HAPPKENDRAHAPPY,"[-0.9939619, -0.9375335, 0.9999368, -0.9818017...",RightTroll
4,HAPPKENDRAHAPPY,"[-0.99595755, -0.9802161, 0.9999972, -0.989883...",RightTroll
...,...,...,...
299995,TESSYELMORE,"[-0.9997602, -0.99756956, 0.9999992, -0.995868...",LeftTroll
299996,TESSYELMORE,"[-0.9988821, -0.9916606, 0.99998736, -0.991392...",LeftTroll
299997,TESSYELMORE,"[-0.9968264, -0.98249024, 0.999998, -0.9969361...",LeftTroll
299998,TESSYELMORE,"[-0.9993973, -0.9954378, 0.99999964, -0.997735...",LeftTroll


## Averaging the vectors for each "author"

In [149]:
list(bert_df_output.groupby('author')['content'].apply(np.mean))

[array([-0.96915984, -0.966579  ,  0.99957913, ..., -0.98325324,
         0.98141277, -0.97459334], dtype=float32),
 array([-0.9841698 , -0.9780887 ,  0.9998793 , ..., -0.9982222 ,
         0.98733264, -0.98026776], dtype=float32),
 array([-0.970223  , -0.97818685,  0.9758654 , ..., -0.9602331 ,
         0.9623901 , -0.9678385 ], dtype=float32),
 array([-0.9424479 , -0.9555731 ,  0.9887826 , ..., -0.9670623 ,
         0.9559627 , -0.96209824], dtype=float32),
 array([-0.8873661 , -0.9190177 ,  0.98100406, ..., -0.9815821 ,
         0.9598946 , -0.9577198 ], dtype=float32),
 array([-0.9932415 , -0.98543584,  0.9999909 , ..., -0.9994138 ,
         0.99405617, -0.9865648 ], dtype=float32),
 array([-0.97932005, -0.9460213 ,  0.9995288 , ..., -0.9868992 ,
         0.97918844, -0.9614875 ], dtype=float32),
 array([-0.96266115, -0.95913106,  0.9939148 , ..., -0.9302393 ,
         0.96309716, -0.9514768 ], dtype=float32),
 array([-0.85814905, -0.8985814 ,  0.9621722 , ..., -0.9464916 ,
       

## Logistic regression

In [151]:
X_BERT = list(bert_df_output.groupby('author')['content'].apply(np.mean))
y = bert_df.drop_duplicates(subset=['author'])['account_category']

In [152]:
X_train, X_test, y_train, y_test = train_test_split(X_BERT, y, test_size=0.1, random_state=0, stratify=y)

In [153]:
lg = LogisticRegression(max_iter=1000)
lg.fit(X_train,y_train)
y_pred = lg.predict(X_test)

In [154]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   LeftTroll       0.33      0.07      0.12        14
    NewsFeed       0.00      0.00      0.00         2
  RightTroll       0.63      0.96      0.76        25

    accuracy                           0.61        41
   macro avg       0.32      0.34      0.29        41
weighted avg       0.50      0.61      0.50        41



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Concatenating U2H and U2M

In [105]:
X = [None] * 917
for i in range(len(X_U2H)):
    X[i] = np.concatenate((X_U2H[i], X_U2M[i]))

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [107]:
lg = LogisticRegression(penalty='l2', max_iter=10000)
lg.fit(X_train,y_train)
y_pred = lg.predict(X_test)

In [108]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

   LeftTroll       0.81      0.73      0.77        66
    NewsFeed       0.76      0.76      0.76        21
  RightTroll       0.90      0.93      0.91       189

    accuracy                           0.87       276
   macro avg       0.82      0.81      0.81       276
weighted avg       0.87      0.87      0.87       276

